# PYNQ tutorial: DMA to streamed interfaces

Overlay consists of two DMAs and an AXI Stream FIFO (input and output AXI stream interfaces). The FIFO represents an accelerator. A single DMA could be used with a read and write channel enabled, but for demonstration purposes, two different DMAs will be used. 

* The first DMA with read channel enabled is connected from DDR to IP input stream (reading from DDR, and sending to AXI stream).
* The second DMA has a write channel enabled and is connected to IP output stream to DDR (receiving from AXI stream, and writing to DDR memory).

There are other IP in the design (not shown) which will be ignored for now.

![](images/dma_stream_example.png)

## 1. Downloading overlay
The overlay can be downloaded automatically when instantiating an overlay class.

In [1]:
from pynq import Overlay

overlay = Overlay("./bitstream/pynq_tutorial.bit")

We can check the IPs in this overlay. Notice the DMAs *axi_dma_from_pl_to_ps* and *axi_dma_from_pl_to_ps*.

In [2]:
overlay.ip_dict

{'axi_dma_from_pl_to_ps': {'addr_range': 65536,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_from_pl_to_ps',
  'gpio': {},
  'interrupts': {'s2mm_introut': {'controller': 'system_interrupts',
    'fullpath': 'axi_dma_from_pl_to_ps/s2mm_introut',
    'index': 1}},
  'phys_addr': 1078001664,
  'state': None,
  'type': 'xilinx.com:ip:axi_dma:7.1'},
 'axi_dma_from_ps_to_pl': {'addr_range': 65536,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_from_ps_to_pl',
  'gpio': {},
  'interrupts': {'mm2s_introut': {'controller': 'system_interrupts',
    'fullpath': 'axi_dma_from_ps_to_pl/mm2s_introut',
    'index': 0}},
  'phys_addr': 1077936128,
  'state': None,
  'type': 'xilinx.com:ip:axi_dma:7.1'},
 'btns_gpio': {'addr_range': 65536,
  'driver': pynq.lib.axigpio.AxiGPIO,
  'fullpath': 'btns_gpio',
  'gpio': {},
  'interrupts': {'ip2intc_irpt': {'controller': 'system_interrupts',
    'fullpath': 'btns_gpio/ip2intc_irpt',
    'index': 4}},
  'phys_addr': 1092681728,
  'state': None,


## 2. Create DMA instances

Using the labels for the DMAs listed above, we can create two DMA objects.

In [3]:
import pynq.lib.dma

dma_send = overlay.axi_dma_from_ps_to_pl
dma_recv = overlay.axi_dma_from_pl_to_ps

## 5. Read DMA
We will read some data from memory, and write to FIFO in the following cells.

The first step is to create the a contiguous memory block. Xlnk will be used to allocate the buffer, and NumPy will be used to specify the type of the buffer. 

In [4]:
from pynq import Xlnk
import numpy as np

data_size = 100

xlnk = Xlnk()

input_buffer = xlnk.cma_array(shape=(data_size,), dtype=np.uint32)

The array can be used like any other NumPy array. We can write some test data to the array. Later the data will be transferred by the DMA to the FIFO. 

In [5]:
for i in range(data_size):
    input_buffer[i] = i + 0xcafe0000

Let's check the contents of the array. The data in the following cell will be sent from PS (DDR memory) to PL (streaming FIFO).

In [6]:
print("Print first few values of buffer ...")
for i in range(10):
    print(hex(input_buffer[i]))

Print first few values of buffer ...
0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


Now we are ready to carry out DMA transfer from a memory block in DDR to FIFO.

In [7]:
dma_send.sendchannel.transfer(input_buffer)

## 5. Write DMA
Let's read the data back from FIFO stream, and write to MM memory. The steps are similar.

We will prepare an empty array before reading data back from FIFO.

In [8]:
output_buffer = xlnk.cma_array(shape=(data_size,), dtype=np.uint32)
print("Print first few values ...")
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

Print first few values ...
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


In [9]:
dma_recv.recvchannel.transfer(output_buffer)

The next cell will print out the data received from PL (streaming FIFO) to PS (DDR memory). This should be the same as the data we sent previously.

In [10]:
print("Print first few values ...")
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

Print first few values ...
0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


## 6. Free all the memory buffers
Don't forget to free the memory buffers to avoid memory leaks!

In [11]:
del input_buffer, output_buffer

## Appendix

## AXI DMA pdf

PDFs can be linked from, or embedded in a Jupyter notebook. The pdf for the AXI DMA documentation is embedded in this notebook below:

In [12]:
from IPython.display import IFrame
IFrame("pg021_axi_dma.pdf", width=800, height=800)